# Tinychat AppView

Collect messages from jetstream, dispatch them. What does an app view do? See
[this](https://github.com/bluesky-social/atproto/discussions/2961)

In [ ]:
//| export

import { Hono } from "hono";
import { upgradeWebSocket } from "hono/deno";
import { startJetstream } from "tinychat/firehose.ts";
import { message } from "@tinychat/ui/message.tsx";
import { createMiddleware } from "hono/factory";
import { TinychatOAuthClient } from "tinychat/oauth.ts";
import { Agent } from "@atproto/api";
import { TinychatAgent } from "tinychat/utils.ts";
import { getDatabase } from "tinychat/db.ts";

In [ ]:
//| export

// based on https://docs.deno.com/examples/chat_app_tutorial/

export default class ChatServer {
  private connectedClients = new Map<string, WebSocket>();

  public handleConnection(ws: WebSocket) {
    const id = `${Math.random() * 100000}`;

    ws.onclose = () => {
      this.clientDisconnected(id);
    };

    this.connectedClients.set(id, ws);
    console.log(">>>>>>> connectedClients", this.connectedClients.size);
  }

  private clientDisconnected(id: string) {
    this.connectedClients.delete(id);
    console.log(`Client ${id} disconnected`);
  }

  public broadcast(message: string) {
    for (const client of this.connectedClients.values()) {
      console.log(">>>>>>> sending message to", client);
      client.send(message);
    }
  }
}

In [ ]:
//| export

const app = new Hono();
const chatServer = new ChatServer();

app.use(
  "*",
  createMiddleware(async (_c, next) => {
    await next();
  }),
);

app.get("/", (c) => c.redirect("https://github.com/callmephilip/tinychat"));

app.get("/__test", (c) =>
  c.html(`<!DOCTYPE html>
<html>
<head>
    <title>HTMX Chat</title>
    <script src="https://unpkg.com/htmx.org@2.0.4"></script>
    <script src="https://unpkg.com/htmx.org/dist/ext/ws.js"></script>
    <style>
        .chat-container { max-width: 600px; margin: 20px auto; }
        .messages { height: 400px; overflow-y: auto; border: 1px solid #ccc; padding: 10px; margin-bottom: 10px; }
        .message { margin: 5px 0; padding: 5px; border-radius: 5px; background: #f0f0f0; }
        .input-form { display: flex; gap: 10px; }
        input { flex-grow: 1; padding: 5px; }
    </style>
</head>
<body>
    <div class="chat-container" hx-ext="ws" ws-connect="/ws">
        <div id="messages" class="messages">
        </div>
        <form class="input-form" ws-send>
            <input type="text" name="message" placeholder="Type a message..." autocomplete="off">
            <button type="submit">Send</button>
        </form>
    </div>
</body>
</html>`));

app.get("/xrpc/chat.tinychat.getServers", async (c) => {
  const authorization = c.req.header("Authorization");

  if (!authorization) {
    return c.json({ error: "Unauthorized" }, 401);
  }

  const ta = await TinychatAgent.create(
    new Agent(
      await TinychatOAuthClient.restoreSessionFromAuthorizationHeader(
        authorization,
      ),
    ),
  );

  await ta.chat.tinychat.server.create(
    { repo: ta.agent.assertDid },
    {
      name: "appview-server",
    },
  );

  return c.json({ message: c.req.header("Authorization") });
});

app.get(
  "/ws",
  upgradeWebSocket(() => {
    return {
      onOpen: (_, ws) => {
        if (!ws.raw) {
          return;
        }
        chatServer.handleConnection(ws.raw);
      },
    };
  }),
);

Hono {
  get: [Function (anonymous)],
  post: [Function (anonymous)],
  put: [Function (anonymous)],
  delete: [Function (anonymous)],
  options: [Function (anonymous)],
  patch: [Function (anonymous)],
  all: [Function (anonymous)],
  on: [Function (anonymous)],
  use: [Function (anonymous)],
  router: SmartRouter { name: "SmartRouter" },
  getPath: [Function: getPath],
  _basePath: "/",
  routes: [
    { path: "/*", method: "ALL", handler: [AsyncFunction (anonymous)] },
    { path: "/", method: "GET", handler: [Function (anonymous)] },
    { path: "/__test", method: "GET", handler: [Function (anonymous)] },
    {
      path: "/xrpc/chat.tinychat.getServers",
      method: "GET",
      handler: [AsyncFunction (anonymous)]
    },
    {
      path: "/ws",
      method: "GET",
      handler: [AsyncFunction: UpgradeWebSocket]
    }
  ],
  errorHandler: [Function: errorHandler],
  onError: [Function: onError],
  notFound: [Function: notFound],
  fetch: [Function: fetch],
  request: [Functi

In [ ]:
//| export

export const runAppView = () => {
  const db = getDatabase();
  console.log("Starting appview with db", db);

  // Cleanup function
  const cleanup = () => {
    console.log("goodbye");
    db.close();
    Deno.exit(0);
  };

  // Handle shutdown signals

  Deno.addSignalListener("SIGINT", cleanup);
  Deno.addSignalListener("SIGTERM", cleanup);

  console.log("Service started");

  startJetstream({
    onMessage: (msg) => {
      console.log(">>>>>>> received message", msg);
      chatServer.broadcast(message(msg).toString());
    },
  });

  Deno.serve(
    { port: parseInt(Deno.env.get("APPVIEW_PORT") || "8000") },
    app.fetch,
  );
};

In [ ]:
const demo = () => {
  runAppView();

  return Deno.jupyter.html`
    <div style="padding: 20px; text-align: center;">
      <iframe
        width="800px"
        height="600px"
        src="http://localhost:8000"
      ></iframe>
    </div>
  `;
};

// Uncomment to run the demo
// demo();